In [ ]:
import os
import cv2

# ✅ 네가 지정한 경로로 수정
fall_video_dir = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/Videos/Fall'  # Fall 영상 폴더
fall_output_dir = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall'  # 저장할 폴더

os.makedirs(fall_output_dir, exist_ok=True)

# 영상별 프레임 추출
for filename in os.listdir(fall_video_dir):
    if not filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        continue

    video_path = os.path.join(fall_video_dir, filename)
    video_name = os.path.splitext(filename)[0]
    save_folder = os.path.join(fall_output_dir, video_name)  # 영상 이름으로 폴더 만들기
    os.makedirs(save_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(save_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"[FALL] {filename} → {frame_count} frames saved in {save_folder}")

[FALL] 00033_H_A_FY_C7.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_A_FY_C7
[FALL] 00033_H_A_FY_C1.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_A_FY_C1
[FALL] 00033_H_A_FY_C5.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_A_FY_C5
[FALL] 00033_H_A_FY_C3.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_A_FY_C3
[FALL] 00033_H_A_FY_C6.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_A_FY_C6
[FALL] 00033_H_A_FY_C4.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_A_FY_C4
[FALL] 00021_H_A_FY_C6.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00021_H_A_FY_C6
[FALL] 00033_H_A_FY_C2.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall/00033_H_

In [ ]:
import os
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from tqdm import tqdm

# ✅ 모델 불러오기
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# ✅ 이미지 전처리 함수
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize_with_pad(img, 256, 256)
    img = tf.cast(img, dtype=tf.int32)
    return tf.expand_dims(img, axis=0)

# ✅ 키포인트 추출 함수
def extract_keypoints(image_path):
    input_image = preprocess_image(image_path)
    outputs = movenet(input_image)
    keypoints_with_scores = outputs['output_0'].numpy()[0, 0, :, :]  # (17, 3)
    return keypoints_with_scores  # [[y, x, confidence], ...]

# ✅ 프레임 폴더 설정
fall_frames_root = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Fall'
csv_output_root = '/content/drive/MyDrive/Colab Notebooks/OpenVideo'
os.makedirs(csv_output_root, exist_ok=True)

# ✅ Fall의 모든 서브폴더 반복 (fall_01, fall_02 등)
for folder in os.listdir(fall_frames_root):
    folder_path = os.path.join(fall_frames_root, folder)
    if not os.path.isdir(folder_path):
        continue

    keypoint_rows = []

    for frame_file in tqdm(sorted(os.listdir(folder_path))):
        if not frame_file.endswith('.jpg'):
            continue
        frame_path = os.path.join(folder_path, frame_file)

        keypoints = extract_keypoints(frame_path)

        row = {'frame': frame_file}
        for i, (y, x, conf) in enumerate(keypoints):
            row[f'keypoint_{i}_x'] = float(x)
            row[f'keypoint_{i}_y'] = float(y)
            row[f'keypoint_{i}_conf'] = float(conf)

        keypoint_rows.append(row)

    # ✅ DataFrame으로 변환 후 CSV 저장
    df = pd.DataFrame(keypoint_rows)
    csv_path = os.path.join(csv_output_root, f'{folder}.csv')
    df.to_csv(csv_path, index=False)
    print(f'✅ {folder} → CSV saved to {csv_path}')

100%|██████████| 600/600 [01:19<00:00,  7.55it/s]


✅ 00033_H_A_FY_C7 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/00033_H_A_FY_C7.csv


100%|██████████| 600/600 [01:14<00:00,  8.10it/s]


✅ 00033_H_A_FY_C1 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/00033_H_A_FY_C1.csv


100%|██████████| 600/600 [01:35<00:00,  6.28it/s]


...

✅ 00021_H_A_FY_C1 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/00021_H_A_FY_C1.csv


100%|██████████| 600/600 [01:51<00:00,  5.40it/s]


✅ 00007_H_A_FY_C5 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/00007_H_A_FY_C5.csv


100%|██████████| 227/227 [00:27<00:00,  8.25it/s]

✅ 00007_H_A_FY_C7 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/00007_H_A_FY_C7.csv


In [ ]:
import os
import cv2

# ✅ 네가 원하는 경로로 수정할 수 있어
normal_video_dir = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/Videos/Normal'  # Normal 영상 폴더
normal_output_dir = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal'  # 저장할 폴더

os.makedirs(normal_output_dir, exist_ok=True)

# 영상별 프레임 추출
for filename in os.listdir(normal_video_dir):
    if not filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        continue

    video_path = os.path.join(normal_video_dir, filename)
    video_name = os.path.splitext(filename)[0]
    save_folder = os.path.join(normal_output_dir, video_name)  # 영상 이름으로 폴더 만들기
    os.makedirs(save_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(save_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"[NORMAL] {filename} → {frame_count} frames saved in {save_folder}")

[NORMAL] 00006_H_A_N_C5.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C5
[NORMAL] 00006_H_A_N_C4.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C4
[NORMAL] 00006_H_A_N_C3.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C3
[NORMAL] 00006_H_A_N_C6.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C6
[NORMAL] 00006_H_A_N_C2.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C2
[NORMAL] 00006_H_A_N_C1.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C1
[NORMAL] 00006_H_A_N_C7.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal/00006_H_A_N_C7
[NORMAL] 00006_H_A_N_C8.mp4 → 600 frames saved in /content/drive/MyDrive/Colab Notebooks/OpenVideo/Frame

In [1]:
import os
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from tqdm import tqdm

# ✅ 모델 로드 (MoveNet Thunder)
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

# ✅ 이미지 전처리 함수
def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize_with_pad(img, 256, 256)
    img = tf.cast(img, dtype=tf.int32)
    return tf.expand_dims(img, axis=0)

# ✅ 키포인트 추출 함수
def extract_keypoints(image_path):
    input_image = preprocess_image(image_path)
    outputs = movenet(input_image)
    keypoints_with_scores = outputs['output_0'].numpy()[0, 0, :, :]  # (17, 3)
    return keypoints_with_scores  # [[y, x, confidence], ...]

# ✅ 프레임 이미지 폴더 & CSV 저장 위치 설정
normal_frames_root = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/Frames/Normal'
csv_output_root = '/content/drive/MyDrive/Colab Notebooks/OpenVideo/CSV files/Normal'
os.makedirs(csv_output_root, exist_ok=True)

# ✅ 모든 normal_XX 폴더 순회
for folder in os.listdir(normal_frames_root):
    folder_path = os.path.join(normal_frames_root, folder)
    if not os.path.isdir(folder_path):
        continue

    keypoint_rows = []

    for frame_file in tqdm(sorted(os.listdir(folder_path))):
        if not frame_file.endswith('.jpg'):
            continue
        frame_path = os.path.join(folder_path, frame_file)

        keypoints = extract_keypoints(frame_path)

        row = {'frame': frame_file}
        for i, (y, x, conf) in enumerate(keypoints):
            row[f'keypoint_{i}_x'] = float(x)
            row[f'keypoint_{i}_y'] = float(y)
            row[f'keypoint_{i}_conf'] = float(conf)

        keypoint_rows.append(row)

    # ✅ DataFrame으로 변환 후 CSV 저장
    df = pd.DataFrame(keypoint_rows)
    csv_path = os.path.join(csv_output_root, f'{folder}.csv')
    df.to_csv(csv_path, index=False)
    print(f'✅ {folder} → CSV saved to {csv_path}')

Exception ignored in: <function _xla_gc_callback at 0x7a10b46d7a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
100%|██████████| 600/600 [02:07<00:00,  4.70it/s]


✅ 00006_H_A_N_C5 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/CSV files/Normal/00006_H_A_N_C5.csv


100%|██████████| 600/600 [02:12<00:00,  4.51it/s]


✅ 00006_H_A_N_C4 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/CSV files/Normal/00006_H_A_N_C4.csv


100%|██████████| 600/600 [01:59<00:00,  5.03it/s]


...

✅ 01849_Y_A_N_C2 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/CSV files/Normal/01849_Y_A_N_C2.csv


100%|██████████| 600/600 [02:13<00:00,  4.49it/s]


✅ 02625_H_A_N_C2 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/CSV files/Normal/02625_H_A_N_C2.csv


100%|██████████| 600/600 [02:12<00:00,  4.54it/s]


✅ 02625_H_A_N_C1 → CSV saved to /content/drive/MyDrive/Colab Notebooks/OpenVideo/CSV files/Normal/02625_H_A_N_C1.csv


In [ ]:
import os
import cv2

# ✅ Suspicious 영상 경로 및 저장 경로 설정
suspicious_video_dir = '/content/drive/MyDrive/my_project/videos/Suspicious'  # suspicious 영상이 있는 폴더
suspicious_output_dir = '/content/drive/MyDrive/my_project/extracted_frames/Suspicious'  # 프레임 저장할 폴더

# 저장 폴더가 없으면 생성
os.makedirs(suspicious_output_dir, exist_ok=True)

# suspicious 폴더 내 모든 영상에 대해 처리
for filename in os.listdir(suspicious_video_dir):
    if not filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        continue  # 영상 파일만 처리

    video_path = os.path.join(suspicious_video_dir, filename)
    video_name = os.path.splitext(filename)[0]
    save_folder = os.path.join(suspicious_output_dir, video_name)
    os.makedirs(save_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(save_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"[SUSPICIOUS] {filename} → {frame_count} frames saved in {save_folder}")